<a href="https://www.kaggle.com/code/abhishridhar/titanic-prediction?scriptVersionId=199122285" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
test=pd.read_csv("/kaggle/input/titanic/test.csv")
train=pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_encoded = pd.get_dummies(train, columns=['Sex', 'Embarked'], drop_first=True)


X_train = train_encoded.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'])
y_train = train_encoded['Survived']

# Handle missing values in 'Age' and 'Fare' (test set will also need the same treatment)
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)



In [4]:
test_encoded = pd.get_dummies(test, columns=['Sex', 'Embarked'], drop_first=True)

missing_cols = set(X_train.columns) - set(test_encoded.columns)
for col in missing_cols:
    test_df_encoded[col] = 0

X_test = test_encoded.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Handle missing values in 'Age' and 'Fare'
X_test_imputed = imputer.transform(X_test)

#Scale
X_test_scaled = scaler.transform(X_test_imputed)

model=RandomForestClassifier(random_state=42)

#Train
model.fit(X_train_scaled, y_train)

# Prediction
y_test_pred = model.predict(X_test_scaled)




**Gradient Boosting Classifier**
=

In [5]:
from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model
clf.fit(X_train_scaled, y_train)

# Make predictions
y_predgb = clf.predict(X_test_scaled)




In [6]:
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],       # Number of boosting stages
    'learning_rate': [0.01, 0.05, 0.1, 0.2],        # Step size shrinkage
    'max_depth': [3, 4, 5, 6, 7],                   # Maximum depth of each tree
    'min_samples_split': [2, 5, 10, 15],            # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4, 6],               # Minimum number of samples required to be at a leaf node
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],         # Fraction of samples used for fitting individual trees
    'max_features': [1.0, 'sqrt', 'log2'],           # Replace 'auto' with 1.0 to keep previous behavior
    'loss': ['log_loss', 'exponential']              # Use 'log_loss' instead of deprecated 'deviance'
}


random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_distributions,
    n_iter=50,  # Number of random parameter settings to try
    cv=5,       # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END learning_rate=0.05, loss=log_loss, max_depth=6, max_features=1.0, min_samples_leaf=2, min_samples_split=15, n_estimators=500, subsample=0.6; total time=   1.8s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=300, subsample=0.6; total time=   0.6s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=300, subsample=0.6; total time=   0.6s
[CV] END learning_rate=0.2, loss=log_loss, max_depth=4, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=200, subsample=0.9; total time=   0.5s
[CV] END learning_rate=0.2, loss=exponential, max_depth=5, max_features=1.0, min_samples_leaf=6, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   0.7s
[CV] END learning_rate=0.01, loss=exponential, max_depth=7, max_features=sqrt, min_samples

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'loss': ['log_loss', 'exponential'],
                                        'max_depth': [3, 4, 5, 6, 7],
                                        'max_features': [1.0, 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42, verbose=2)

In [7]:
rfmodelgbrandom = random_search.best_estimator_

rfmodelgbrandom.fit(X_train_scaled, y_train)

# Make predictions on the test data
ypredgbrandom = rfmodelgbrandom.predict(X_test_scaled)

**XGB**
=

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [9]:
# model1=XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# model1.fit(X_train_scaled, y_train)
# y_test_pred1 = model.predict(X_test_scaled)

**Random Search**
=

**Let's try tuning the model.**

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt','auto'],
#     'bootstrap': [True, False]
# }


from sklearn.model_selection import RandomizedSearchCV

# rf = RandomForestClassifier(random_state=42,class_weight='balanced') 

# # Setup RandomizedSearchCV
# random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, 
#                                    n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=2)


# # Fit the random search to the data
# random_search.fit(X_train_imputed, y_train)

In [12]:
# rfmodelforestrandom = random_search.best_estimator_

# # Define a pipeline
# pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),  # Or 'median' depending on your needs
#     ('model', rfmodelforestrandom)  # Your best estimator from RandomizedSearchCV
# ])

# # Fit the model on the training data
# pipeline.fit(X_train, y_train)

# # Make predictions on the test data
# ypredforestrandom = pipeline.predict(X_test)

**Grid Search**
=

In [13]:
# param_grid = {
#     'n_estimators': [50, 100, 200],  
#     'max_depth': [None, 10, 20, 30],      
#     'min_samples_split': [2, 5, 10],      
#     'min_samples_leaf': [1, 2, 4],        
#     'max_features': ['log2', 'sqrt'],     
#     'bootstrap': [True, False]            
# }


# from sklearn.model_selection import GridSearchCV
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)




# grid_search.fit(X_train_imputed, y_train)



In [14]:
# rfmodelforestgrid = grid_search.best_estimator_

# pipeline2 = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),  # Or 'median' depending on your needs
#     ('model', rfmodelforestgrid)  # Your best estimator from RandomizedSearchCV
# ])


# pipeline2.fit(X_train,y_train)
# ypredforestgrid = pipeline2.predict(X_test)

**Submission**
=

In [15]:
submission=pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived':ypredgbrandom
})

In [16]:
submission.to_csv('gender_submission.csv',index=False)

In [17]:
# submission = pd.DataFrame({
#     'PassengerId': test['PassengerId'],
#     'Survived': y_test_pred
# })

In [18]:
# submission.to_csv('gender_submission.csv', index=False)